<h1>Прогнозирование оттока клиентов телеком-оператора ТелеДом</h1>

Оператор связи «ТелеДом» хочет бороться с оттоком клиентов. Для этого его сотрудники начнут предлагать промокоды и специальные условия всем, кто планирует отказаться от услуг связи. Чтобы заранее находить таких пользователей, «ТелеДому» нужна модель, которая будет предсказывать, разорвёт ли абонент договор. Команда оператора собрала персональные данные о некоторых клиентах, информацию об их тарифах и услугах. Задача — обучить на этих данных модель для прогноза оттока клиентов.


<b>Цели и задачи проекта</b>

Основная цель проекта – разработать модель машинного обучения, которая сможет предсказывать вероятность оттока клиентов телеком-оператора TeleDom на основе анализа их поведения и использования услуг.


Задачи проекта включают:


- Анализ предоставленных данных о клиентах и их использовании услуг.<br>
- Предобработка данных для подготовки к обучению модели.<br>
- Выбор и обучение модели машинного обучения для прогнозирования оттока. <br>
- Оценка эффективности модели с использованием соответствующих метрик.<br>

Критерии успеха:<br>
 Проект будет считаться успешным, если достигнуты следующие показатели:

- AUC-ROC выше 0.85.

# **Шаг 1. Загрузка данных**






In [ ]:
!pip install pyspark
!pip install phik
!pip install catboost
!pip install --upgrade scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import phik
import sklearn
import pickle
from sklearn.preprocessing import OneHotEncoder
from pyspark.sql import SparkSession
from phik import resources, report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix


In [ ]:
RANDOM_STATE=190224

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#load datasets
contract = pd.read_csv('/content/drive/MyDrive/datasets/contract_new.csv')

display(contract.head())

In [ ]:
#size of dataset contract
contract.shape

In [ ]:
contract.info()

In [ ]:
print(contract.isnull().sum())

In [ ]:
duplicate_count = contract.duplicated(subset='customerID', keep=False).sum()
print(duplicate_count)


Первичный анализ данных был проведен в наборе данных «контракт». 7043 строки, 8 столбцов. Некоторые типы данных следует изменить, чтобы они лучше соответствовали содержащейся в них информации. Данные следует проверить на наличие явных и скрытых дубликатов.

In [ ]:
#load dataset
internet = pd.read_csv('/content/drive/MyDrive/datasets/internet_new.csv')
display(internet.head())

In [ ]:
#size of dataset internet
internet.shape

In [ ]:
internet.info()

In [ ]:
print(internet.isnull().sum())

In [ ]:
duplicate_count = internet.duplicated(subset='customerID', keep=False).sum()
print(duplicate_count)

Первичный анализ данных был проведен в наборе данных «Интернет». 5517 строк, 8 столбцов. Некоторые типы данных следует изменить, чтобы они лучше соответствовали содержащейся в них информации.

In [ ]:
personal = pd.read_csv('/content/drive/MyDrive/datasets/personal_new.csv')
display(personal.head())

In [ ]:
#size of dataset personal
personal.shape

In [ ]:
personal.info()

In [ ]:
print(personal.isnull().sum())

In [ ]:
duplicate_count = personal.duplicated(subset='customerID', keep=False).sum()
print(duplicate_count)

Первичный анализ данных был проведен в «personal» наборе данных. 7043 строки, 5 столбцов. Некоторые типы данных следует изменить, чтобы они лучше соответствовали содержащейся в них информации. Данные следует проверить на наличие явных и скрытых дубликатов.

In [ ]:
phone = pd.read_csv('/content/drive/MyDrive/datasets/phone_new.csv')
display(phone.head())

In [ ]:
#size of dataset phone
phone.shape

In [ ]:
phone.info()

In [ ]:
print(phone.isnull().sum())

In [ ]:
duplicate_count = phone.duplicated(subset='customerID', keep=False).sum()
print(duplicate_count)

Первоначальный анализ данных был проведен в наборе данных «Phone». 6361 строка, 2 столбца.Пропущенные данные отсутствуют. Дубликаты по customerID отсуствуют

Данные из 4 наборов данных были успешно загружены. Данные необходимо предварительно обработать. Это будет сделано в следующем разделе.

# **Шаг 2. Исследовательский анализ данных и предобработка**

## Датасет Contract















В данном датасете необходимо выполнить следующие шаги для исследовательского анализа данных и предобработки:<br>
-Перевод названия колонок в нижний регистр, так как это является общепринятой практикой.  
-Поиск явных дубликатов  
-Поиск пропущенных данных  
-Удаление явных дубликатов, если они есть  
-Перевод формата столбцов с датами в формат datetime64  
-Перевод totalcharges в числовой формат  
-Если в этой колонке будут найдены  пропуски - это будет означать, что существуют клиенты, которые еще не успели внести ни одну плату, и следовательно эти данные можно будет заменить 0  
-Необходимо завести целевую переменную - отток клиентов - если они разорвали контракт в колонке enddate будет стоять дата окончания контракта   
-Если дата окончания контракта не стоит, следовательно клиент продолжает пользоваться услугами, и можно заменить пропущенные данные на дату 2020-02-01 - контракты актуальны до этой даты.
-Для числовых данных провести анализ на разброс данных и выбросы с помощью графика Boxplot  
-Провести анализ распределения целевой переменной Churn.





In [ ]:
#converting column names to lowercase
contract.columns = contract.columns.str.lower()

In [ ]:
#looking for obvious duplicates
duplicates = contract.duplicated()
print(duplicates.sum())

In [ ]:
#delete dublicates
contract = contract.drop_duplicates()
contract.info()

In [ ]:
# Convert 'begindate' and 'enddate' to datetime
contract['begindate'] = pd.to_datetime(contract['begindate'])
contract['enddate'] = pd.to_datetime(contract['enddate'], errors='coerce')

# Convert 'totalcharges' to numeric, errors='coerce' will replace non-numeric values with NaN
contract['totalcharges'] = pd.to_numeric(contract['totalcharges'], errors='coerce')

# Check the new data types
print(contract.info())

In [ ]:
# Replacing all types of missing values with 0
contract['totalcharges'].fillna(0, inplace=True)

Найденные пропуски totalcharges  предположительно означают,что существуют клиенты, которые еще не успели внести ни одну плату, и следовательно эти данные можно будет заменить 0

In [ ]:
# Creating a target column 'churn' indicating whether a customer has churned
# 1 if 'end_date' has a date (the contract was terminated), 0 otherwise
contract['churn'] = contract['enddate'].notna().astype(int)

# Checking the distribution of the target variable
print(contract['churn'].value_counts())

In [ ]:
contract.info()

In [ ]:
#replace missing end_date because the information in contracts is actual  to 2020-02-01
replacement_date = pd.Timestamp('2020-02-01')
contract['enddate'] = contract['enddate'].fillna(replacement_date)

In [ ]:
contract.head(3)

In [ ]:
# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Creating boxplots for 'monthlycharges' and 'totalcharges'
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.boxplot(x=contract['monthlycharges'])
plt.title('Boxplot of Monthly Charges')

plt.subplot(1, 2, 2)
sns.boxplot(x=contract['totalcharges'])
plt.title('Boxplot of Total Charges')

plt.show()

Был проведен анализ аномалий и выбросов, при этом особое внимание было уделено столбцу TotalCharges. В результате анализа стало ясно, что в данных наблюдаются значительные разбросы, что может указывать на наличие долгосрочных клиентов с высокими общими затратами.

In [ ]:
columns = ['type', 'paperlessbilling', 'paymentmethod', 'churn']

fig, axes = plt.subplots(nrows=len(columns), ncols=1, figsize=(10, 20))

for i, col in enumerate(columns):

    value_counts = contract[col].value_counts(normalize=True) * 100
    value_counts.plot(kind='bar', ax=axes[i], title=col, color='skyblue')
    axes[i].set_ylabel('Percentage (%)')
    axes[i].set_xlabel('')

plt.tight_layout()
plt.show()

Среди типов оплаты контракта преобладает  -ежемесячный

Электронный тип оплаты преобадает,остальные 3 типа оплаты занимают примерно одинаковые пропорции

Электронный расчётный лист запрашивает больше половины клиентов.

In [ ]:
contract['churn'].describe()

В рамках раздела «Исследовательский анализ и предварительная обработка данных» для набора данных «Contract» были выполнены следующие действия и сделаны соответствующие выводы:

Предварительная проверка данных. Набор данных был тщательно проверен на предмет типов данных, пропущенных значений и общего содержания. Было обнаружено, что данные содержат как числовые, так и категориальные характеристики, а также отсутствующие значения в столбце EndDate.

Обработка пропущенных значений. Отсутствующие значения в EndDate интерпретировались как индикаторы текущих контрактов (клиенты продолжают пользоваться услугами). Эти значения заменены на дату 01.02.2020 - данные в договорах актуальны до этой даты.

Анализ и обработка аномалий. Был проведен анализ аномалий и выбросов, при этом особое внимание было уделено столбцу TotalCharges. В результате анализа стало ясно, что в данных наблюдаются значительные разбросы, что может указывать на наличие долгосрочных клиентов с высокими общими затратами.

Среди типов оплаты контракта преобладает -ежемесячный.Электронный расчётный лист запрашивает больше половины клиентов. Электронный тип оплаты преобадает.

Анализ целевой переменной: был проведен анализ распределения целевой переменной (отток клиентов), который показал, что около 15,63% клиентов расторгли договор. Это обеспечивает основу для дальнейшего построения модели прогнозирования оттока клиентов.

## Датасет **Internet**

В данном датасете необходимо выполнить следующие шаги для исследовательского анализа данных и предобработки:<br> -Перевод названия колонок в нижний регистр, так как это является общепринятой практикой.   
-Поиск пропущенных данных  
-Дубликаты в данном датасете не требуют удаления,так как данные в колонках не носят персонафицированный характер и могут быть одинакомыми для нескольких клиентов   
-Проанализировать  распределение категориальных данных

In [ ]:
internet.head()

In [ ]:
#converting column names to lowercase
internet.columns = internet.columns.str.lower()

In [ ]:
internet.info()

In [ ]:
columns = ['internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
           'techsupport', 'streamingtv', 'streamingmovies']

fig, axes = plt.subplots(nrows=len(columns), ncols=1, figsize=(10, 20))

for i, col in enumerate(columns):

    value_counts = internet[col].value_counts(normalize=True) * 100
    value_counts.plot(kind='bar', ax=axes[i], title=col, color='skyblue')
    axes[i].set_ylabel('Percentage (%)')
    axes[i].set_xlabel('')

plt.tight_layout()
plt.show()

Использование оптоволокна: примерно 56,12% пользователей используют оптоволоконное подключение к Интернету. Это свидетельствует о высокой популярности оптоволокна среди рассматриваемой выборки пользователей.

Интернет-безопасность: около 36,60% пользователей подписываются на службу онлайн-безопасности. Это показывает, что услуга не так популярна, как могла бы быть, возможно, из-за дополнительной стоимости или недостаточной осведомленности пользователей о преимуществах.

Онлайн-резервное копирование. Службой онлайн-резервного копирования пользуются примерно 44,03% пользователей. Это говорит об умеренной популярности сервиса.

Защита устройства. Примерно 43,90% пользователей воспользовались услугой защиты устройств, что практически идентично популярности онлайн-резервного копирования.

Техподдержка: услугами технической поддержки пользуются около 37,05% пользователей, что делает ее сравнительно менее популярной среди предлагаемых услуг.

Интернет-телевидение (потоковое телевидение): почти половина пользователей (49,07%) используют сервис потокового телевидения, что делает его одним из наиболее популярных сервисов среди рассмотренных.

Фильмы через Интернет (Streaming Movies): Чуть более популярным, чем потоковое телевидение, является сервис потокового кино, которым пользуются 49,52% пользователей.

Вывод: в целом данные показывают, что услуги, связанные с развлечениями (такие как потоковое телевидение и фильмы), более популярны, чем услуги безопасности и поддержки (такие как онлайн-безопасность и техническая поддержка). Оптоволоконные подключения к Интернету также очень популярны, что может указывать на желание пользователей иметь высокоскоростное соединение для более требовательных услуг, таких как потоковое видео.

**Summary of Step 2: Exploratory Data Analysis and Preprocessing (Dataset "Internet")**



Анализ показал, что, хотя большинство клиентов предпочитают оптоволоконное соединение, другие услуги, такие как онлайн-резервное копирование и потоковая передача, также пользуются значительной популярностью.
Отсутствие спроса на некоторые дополнительные услуги, такие как онлайн-безопасность и защита устройств, может иметь решающее значение при принятии решений относительно стратегии предоставления услуг и маркетинговой политики.
Предлагаемые услуги могут быть оптимизированы на основе предпочтений клиентов, выявленных в результате анализа данных, с целью повышения удовлетворенности клиентов и сокращения их оттока.

## Датасет **Personal**















В данном датасете необходимо выполнить следующие шаги для исследовательского анализа данных и предобработки:<br> -Перевод названия колонок в нижний регистр, так как это является общепринятой практикой.  
-Поиск пропущенных данных  
-Дубликаты в данном датасете не требуют удаления,так как данные в колонках не носят персонафицированный характер и могут быть одинакомыми для нескольких клиентов  
-Проанализировать распределение категориальных данных  

In [ ]:
#converting column names to lowercase
personal.columns = personal.columns.str.lower()

In [ ]:
#looking for obvious duplicates
personal.info()

In [ ]:
personal.head()

In [ ]:
# Viewing unique values for each column
for column in personal.columns:
    print(f"Unique values in column {column}: {personal[column].unique()}")

In [ ]:
personal.info()

In [ ]:
columns = ['gender', 'seniorcitizen', 'partner', 'dependents']

fig, axes = plt.subplots(nrows=len(columns), ncols=1, figsize=(10, 20))

for i, col in enumerate(columns):

    value_counts = personal[col].value_counts(normalize=True) * 100
    value_counts.plot(kind='bar', ax=axes[i], title=col, color='skyblue')
    axes[i].set_ylabel('Percentage (%)')
    axes[i].set_xlabel('')

plt.tight_layout()
plt.show()

Пожилые люди: примерно 16,2% подписчиков — пожилые люди. Это свидетельствует о том, что большая часть абонентов (около 83,8%) не попадает в категорию старшего поколения.

Пол: Распределение по полу показывает, что примерно 50,5% подписчиков составляют мужчины, что означает почти равное распределение мужчин и женщин среди подписчиков. Это свидетельствует о хорошем гендерном балансе в выборке.

Партнер: Примерно 48,3% подписчиков указали, что у них есть партнер. Это говорит о том, что распределение между подписчиками с партнерами и без партнеров вполне равномерное, с небольшим уклоном в сторону отсутствия партнера.

Иждивенцы: только около 30% подписчиков имеют иждивенцев. Это показывает, что у большинства подписчиков нет иждивенцев, что может указывать на молодую аудиторию или аудиторию, состоящую из людей среднего возраста, не имеющих детей или других иждивенцев.

На основе исследовательского анализа данных и предварительной обработки «Personal» набора данных:

Набор данных состоит из 7043 записей с четырьмя столбцами: «пол», «пожилой гражданин», «партнер» и «иждивенцы».

Информация о визуализации: Благодаря визуализации мы получили ценную информацию о составе демографической группы набора данных. Например, доля пожилых людей в наборе данных составляет примерно 16,2%, что указывает на меньшинство населения с потенциально разными потребностями и предпочтениями. Распределение по гендерному признаку почти сбалансировано: незначительное большинство клиентов-мужчин. Данные о статусе партнерства и зависимости показывают разнообразную клиентскую базу: менее половины клиентов имеют партнеров или иждивенцев. Эти идеи могут сыграть важную роль в адаптации услуг и маркетинговых стратегий для удовлетворения различных потребностей клиентской базы.

## Датасет **Phone**

В данном датасете необходимо выполнить следующие шаги для исследовательского анализа данных и предобработки: -Перевод названия колонок в нижний регистр, так как это является общепринятой практикой.  
-Поиск пропущенных данных  
-Дубликаты в данном датасете не требуют удаления,так как данные в колонках не носят персонафицированный характер и могут быть одинакомыми для нескольких клиентов  
-Проанализировать распределение категориальных данных  

In [ ]:
#converting column names to lowercase
phone.columns = phone.columns.str.lower()

In [ ]:
phone.info()

In [ ]:
phone.head()

In [ ]:
value_counts = phone['multiplelines'].value_counts(normalize=True) * 100

# Visualisation
plt.figure(figsize=(8, 6))
value_counts.plot(kind='bar', color='skyblue')
plt.title('Multiple Lines Option Distribution')
plt.ylabel('Percentage (%)')
plt.xlabel('Multiple Lines Option')
plt.xticks(rotation=45)
plt.show()

Данные о функции нескольких линий в наборе телефонных данных иллюстрируют сбалансированное распределение предпочтений клиентов в отношении услуг нескольких линий с небольшим уклоном в сторону отсутствия нескольких линий. Эта информация может быть полезна для понимания потребностей клиентов и соответствующей адаптации услуг или маркетинговых стратегий.

**Вывод по исследовальскому анализу данных (Датасет "Phone")**

Набор данных состоит из 6361 записи с столбцами: «multiplelines».

Столбец «multiplelines» указывает, есть ли у клиента несколько строк (Да) или нет (Нет). Набор данных показывает, что примерно 53,29% клиентов не имеют нескольких линий, а около 46,71% имеют несколько линий. Эта информация дает представление о распределении функций телефонных услуг среди клиентов, что может быть полезно для понимания предпочтений клиентов и оптимизации предложений услуг.


# **Шаг 3 Объединение данных**

In [ ]:
# Merge contract с personal
merged_df = contract.merge(personal, on='customerid', how='outer')

# Add data from  internet
merged_df = merged_df.merge(internet, on='customerid', how='outer')

# Add data from phone
final_df = merged_df.merge(phone, on='customerid', how='outer')

Для объединения данных в данном случае лучше всего выбрать  объединение outer  так как этот тип объединения позволяет сохранить полную информацию по всем клиентам, включая тех, кто может быть только в одном из датасетов. Это важно для комплексного анализа клиентской базы, где некоторые клиенты могут пользоваться только определенными услугами.

In [ ]:


# Display the first few rows of the DataFrame
display(final_df .head())

In [ ]:
final_df .info()

Всего 7043 записи, что соответствует исходному количеству клиентов в датасете contract. Это указывает на то, что каждая запись в датасете contract нашла соответствие хотя бы в одном из других датасетов, или же все записи из contract были сохранены в результате объединения типом outer.  
Присутствие пропусков в данных: Пропуски в столбцах, связанных с интернет-услугами (internetservice, onlinesecurity, onlinebackup, deviceprotection, techsupport, streamingtv, streamingmovies) и телефонными услугами (multiplelines), говорят о том, что не все клиенты пользуются этими услугами. Так, интернет-услугами пользуются 5517 клиентов, а услугами телефонии — 6361 клиент. <br>Для дальнейшего анализа необходимо ввести новую переменную -'Длительность контракта' - это позволит понять насколько долго клиенты остаются с компанией.А для дальнейшей работы с данными предполагается создать новый датафрейм и добавить в него только необходимые для анализа колонки.

In [ ]:
duplicate_count = final_df.duplicated(subset='customerid', keep=False).sum()
print(duplicate_count)

In [ ]:
# #assigning a column index
# final_df.set_index('customerid', inplace=True)

In [ ]:
final_df.info()

In [ ]:
# Calculate the duration of the contract and adding new column
final_df['contract_duration'] = (final_df['enddate'] - final_df['begindate']).dt.days

Была вычислена длительность контракта каждого клиента в днях, вычитая дату начала контракта (begindate) из даты его окончания (enddate). Результат был сохранен в новом столбце contract_duration. Это позволяет анализировать, насколько долго клиенты остаются с компанией, что является важным показателем для понимания лояльности клиентов и оттока

In [ ]:
final_df.head()

In [ ]:
final_df.info()

In [ ]:
cols = final_df.select_dtypes(include=['object','float']).columns.tolist()
# add manually identified columns
additional_cols = ['churn','contract_duration', 'seniorcitizen']

# Combine lists while ensuring uniqueness
cols = list(set(cols + additional_cols))
df = final_df[cols].copy()

В данной ячейке был сформирован список столбцов для дальнейшего  анализа, включая категориальные и вещественные переменные, а также важные идентифицированные нами столбцы churn, contract_duration. Это позволяет сфокусироваться на ключевых данных, подготовить их к анализу и моделированию, обеспечивая при этом уникальность и актуальность выборки столбцов

In [ ]:
df.info()

## **Вывод**


Наборы данных успешно объединены. После получения комбинации данных необходимо провести предварительную обработку данных. Для этого будут найдены и при необходимости удалены дубликаты. Поскольку большой процент клиентов не подключены к сети, данные о них отключены в Интернете. Все недостающие данные из-за этого будут иметь значение новое введенное значенин Not Applicable. Что касается мультилиний, предположение то же - из-за отсутствия в таблице того, что у клиентов нет мультистрок - пропущенные значения будут заполнены также Not Applicable.

# **Шаг  4. Исследовательский анализ данных и предобработка объединенного датафрейма**

In [ ]:
#looking for obvious duplicates
duplicates = df.duplicated(keep=False)


display(df[duplicates])
print(duplicates.sum())

In [ ]:
# counting persent of dublicates
duplicates = df.duplicated(keep=False).sum()
percentage_duplicates = (duplicates / len(df)) * 100

print(f"persent of dublicates: {percentage_duplicates}%")

In [ ]:
# fillimg missing values to multiplelines
df['multiplelines'].fillna('Not Applicable', inplace=True)

# check results
print(df[[ 'multiplelines']].info())

In [ ]:
# Assume missing values indicate no internet service

internet_service_columns = ['internetservice',
    'onlinesecurity', 'onlinebackup', 'deviceprotection',
    'techsupport', 'streamingtv', 'streamingmovies'
]
# Fill missing values

# For columns indicating presence of a service, fill missing values with Not Applicable (service is not provided)
df[internet_service_columns] = df[internet_service_columns].fillna('Not Applicable')


df.info()

In [ ]:
# Visualisation of numeric features
num_features = ['monthlycharges', 'totalcharges', 'contract_duration']
for feature in num_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(df[feature], kde=True, bins=30)
    plt.title(f'distribution {feature}')
    plt.xlabel(feature)
    plt.ylabel('amount')
    plt.show()

Глядя на график, можно сказать, что для обоих столбцов в начале графика наблюдается рост. для ежемесячных начислений это значение около 20, а для общих начислений близко к 0. для ежемесячных начислений происходит плавное увеличение суммы до 80, а затем плавное уменьшение
Для столбца длительность контракта - характерно большое количество краткосрочных контрактов - затем постепенное уменьшение и небольшой пик на значениях около 2500

In [ ]:
df.info()

In [ ]:
# Visualisation of numeric features
num_features = ['monthlycharges', 'totalcharges','contract_duration' ]
for feature in num_features:
    plt.figure(figsize=(10, 6))
    sns.histplot(df[feature], kde=True, bins=30)
    plt.title(f'distribution {feature}')
    plt.xlabel(feature)
    plt.ylabel('amount')
    plt.show()

In [ ]:
#Distribution of Churn
plt.figure(figsize=(8, 5))
sns.countplot(x='churn', data=df)
plt.title('Distribution of Churn')
plt.show()

Для целевой переменной существует большой дисбаланс классов. Однако выбранная метрика оценки — площадь под ROC-кривой — является устойчивой к дисбалансу классов.  Это делает ROC-AUC подходящей метрикой  позволяя сосредоточиться на улучшении способности модели различать классы, а не только на точности предсказаний.

In [ ]:
df_for_phik = df.drop(columns=['customerid'])

# Specify your interval columns. For example, 'monthlycharges' and 'totalcharges' are numerical.
interval_columns = ['monthlycharges', 'totalcharges', 'contract_duration']

# Calculate the phik correlation matrix
phik_matrix = df_for_phik.phik_matrix(interval_cols=interval_columns)

# Extract the 'churn' column to get correlations with 'churn' only
churn_correlation = phik_matrix['churn'].sort_values(ascending=False)


churn_correlation = churn_correlation.drop('customerid', errors='ignore')

# Print the correlation of 'churn' with all other variables
print(churn_correlation)

# # Visualize the phik correlation matrix
# plt.figure(figsize=(12, 10))
# sns.heatmap(phik_matrix.drop(index=['customerid'], columns=['customerid'], errors='ignore'), annot=True, fmt=".2f", cmap='coolwarm', square=True)
# plt.title("Phik Correlation Matrix")
# plt.show()

In [ ]:
# display(phik_matrix)

<!-- Totalcharges имеет заметную положительную корреляцию с онлайн защитой,онлайн бэкап,стриминговые сервисами,и стримингТВ что указывает на то, что клиенты с этими услугами, как правило, имеют более высокие общие расходы.

Статус пенсионера коррелирует заметно с иждивенцами, что логично и также с ежемесячными тратами.

Подключенные мул

онлайн-безопасность заметно коррелирует с технической поддержкой (0,527941), что позволяет предположить, что клиенты, выбирающие услуги онлайн-безопасности, также склонны пользоваться технической поддержкой.

Способ оплаты «Электронный чек» имеет заметную корреляцию с безбумажным выставлением счетов (0,321028), что может означать, что клиенты, которые платят с помощью электронных чеков, с большей вероятностью выберут безбумажный счет.

Contract_duration показывает очень сильную корреляцию с общими расходами (0,848304), указывая на то, что более длительные контракты связаны с более высокими общими расходами.

Internet_service имеет высокую корреляцию с несколькими функциями, такими как защита устройства (0,562002), техническая поддержка (0,502876) и онлайн-резервное копирование (0,563092), что позволяет предположить, что тип интернет-услуги влияет на внедрение этих дополнительных услуг.


Ежемесячные расходы и потоковое телевидение (0,835016) демонстрируют очень сильную корреляцию, указывающую на то, что клиенты, пользующиеся услугами потокового телевидения, как правило, имеют более высокие ежемесячные расходы.

В строке и столбце оттока показаны корреляции с различными признаками, но ни один из них не является чрезвычайно высоким, что указывает на то, что ни один признак сам по себе не является окончательным предиктором оттока. Эта матрица полезна для выявления взаимосвязей между различными атрибутами и поведением клиентов, которые могут служить основой для бизнес-стратегий, маркетинга, обслуживания клиентов и усилий по их удержанию.

Продолжительность контракта (0,375). Значительная положительная корреляция предполагает, что более длительный срок контракта связан с более высоким уровнем оттока клиентов. Это может указывать на проблемы с обязательствами клиентов или неудовлетворенность ими с течением времени.

Общие расходы (0,303): демонстрирует умеренную положительную корреляцию с оттоком. Более высокие общие расходы могут привести к увеличению оттока клиентов, возможно, из-за воспринимаемой ценности или финансовых ограничений с точки зрения клиента.

Партнер (0,227) и Ежемесячные платежи (0,226): оба демонстрируют одинаковый уровень положительной корреляции с оттоком. Наличие партнера и более высокие ежемесячные платежи умеренно способствуют оттоку клиентов. Это может отразиться на финансовом бремени и динамике принятия решений в домохозяйствах.

Способ оплаты (0,215): умеренная корреляция указывает на то, что выбор способа оплаты влияет на отток клиентов. Определенные методы могут быть менее удобными или менее предпочтительными, что влияет на удовлетворенность и удержание клиентов.

Несколько линий (0,105): более низкая положительная корреляция предполагает, что наличие нескольких линий незначительно влияет на отток клиентов. Это может указывать на сложности или уровень удовлетворенности масштабируемостью услуги.

Тип (0,094), онлайн-резервное копирование (0,090) и потоковая передача фильмов (0,087): эти функции демонстрируют меньшую корреляцию с оттоком, что указывает на умеренное влияние типа услуги, вариантов резервного копирования данных и развлекательных функций на удержание клиентов. «Пожилой гражданин» (0,086), «Защита устройств» (0,086) и «Безбумажный биллинг» (0,083): указывают на очень слабую положительную корреляцию с оттоком, что позволяет предположить, что демографические факторы и дополнительные услуги играют незначительную роль во влиянии на решения об оттоке.

Потоковое телевидение (0,079), Онлайн-безопасность (0,059), Интернет-услуги (0,057) и Техподдержка (0,052): эти функции демонстрируют наименьшую положительную корреляцию, что позволяет предположить, что они оказывают наименьшее влияние на отток среди перечисленных факторов. Их влияние на решение клиентов отказаться от услуги минимально, но немаловажно.

Иждивенцы (0,047): указывает на низкую корреляцию.

Пол (0,009): показывает почти незначительную корреляцию с оттоком, указывая на то, что пол не оказывает существенного влияния на вероятность оттока.

Это резюме дает ценную информацию о факторах, влияющих на отток клиентов. Более высокие затраты (как ежемесячные, так и общие), менее гибкие способы оплаты и более длительный срок действия контрактов, как правило, увеличивают риск оттока клиентов. -->

In [ ]:
churn_correlation_df = phik_matrix[['churn']].drop(index=['customerid'], errors='ignore')

# Визуализация тепловой карты для корреляций churn
plt.figure(figsize=(4, 8)) # Размер фигуры адаптирован под один столбец
sns.heatmap(churn_correlation_df, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title("Phik Correlation with Churn")
plt.show()

Анализируя корреляцию  между переменной "churn" и другими признаками, можно сделать следующие выводы относительно влияния различных факторов на отток клиентов:

- **Продолжительность контракта** (\(0.374569\)) имеет умеренную корреляцию с оттоком, что подчеркивает, что клиенты с более короткими контрактами склонны к увеличению оттока. Это может указывать на то, что более длительные контракты связаны с лояльностью клиентов.

- **Общие расходы** (\(0.302890\)) также показывают умеренную корреляцию с оттоком. Это может свидетельствовать о том, что высокие общие расходы могут способствовать решению клиентов об уходе, возможно, из-за восприятия недостаточной ценности услуг.

- **Наличие партнера** (\(0.226688\)) и **ежемесячные расходы** (\(0.225925\)) имеют схожие  слабые уровни корреляции с оттоком, что указывает на то, что эти факторы могут немного влиять на решения клиентов об уходе, хотя и в меньшей степени по сравнению с продолжительностью контракта и общими расходами.

- **Способ оплаты** (\(0.214832\)) слабо  влияет на отток

- Признаки, такие как **многоканальная связь (multiplelines)** (\(0.105101\)), **тип контракта (type)** (\(0.094015\)), и **онлайн-резервное копирование (onlinebackup)** (\(0.089704\)), показывают более слабую корреляцию с оттоком, что указывает на их менее выраженное влияние на решение об уходе.

- **Демографические признаки**, такие как **возраст (seniorcitizen)** (\(0.086159\)), и **пол (gender)** (\(0.008581\)), имеют минимальное влияние на отток, что подчеркивает, что решения об уходе определяются скорее условиями предоставления услуг, чем демографическими факторами.

Эти выводы подчеркивают значимость определенных услуг и условий контракта для удержания клиентов и могут служить основой для разработки стратегий по снижению оттока, например, путем предложения более привлекательных условий для долгосрочных контрактов, оптимизации ценообразования и улучшения условий оплаты.

In [ ]:
# Разделение датасета на две части: ушли (churn = 1) и остались (churn = 0)
df_churned = df[df['churn'] == 1]
df_stayed = df[df['churn'] == 0]

# Вычисление матрицы корреляции phi_k для клиентов, которые ушли
phik_matrix_churned = df_churned.phik_matrix(interval_cols=interval_columns)

# Вычисление матрицы корреляции phi_k для клиентов, которые остались
phik_matrix_stayed = df_stayed.phik_matrix(interval_cols=interval_columns)

In [ ]:
display(phik_matrix_churned )

In [ ]:
# Визуализация матрицы корреляции phi_k для клиентов, которые ушли
plt.figure(figsize=(12, 10))
sns.heatmap(phik_matrix_churned, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Phik Correlation Matrix for Churned Customers")
plt.show()


Из  таблицы корреляции между различными признаками и оттоком клиентов (churn) можно сделать следующие выводы:

1. **Продолжительность контракта** (\(0.837747\)) имеет высокую корреляцию с общими расходами (totalcharges), что указывает на то, что клиенты с более длительными контрактами склонны иметь более высокие общие расходы. Это может свидетельствовать о более высокой лояльности и удовлетворенности услугами.

2. **Интернет-услуга** (\(0.926400\)) и **ежемесячные платежи** (\(1.000000\)) показывают весьма высокую корреляцию, подчеркивая, что выбор интернет-услуги значительно влияет на размер ежемесячных платежей клиентов.

3. **Онлайн-безопасность** (\(0.818126\)), **техподдержка** (\(0.824158\)), и **защита устройства** (\(0.821914\)) также демонстрируют высокую корреляцию с ежемесячными платежами, что может указывать на то, что клиенты, использующие эти услуги, склонны платить больше каждый месяц.

4. Важно отметить, что **пол** (gender) и **наличие партнера** (partner) показывают нулевую или очень слабую корреляцию с большинством других признаков, что говорит о том, что эти демографические признаки имеют ограниченное влияние на использование и оплату услуг.

5. **Стриминговые сервисы** (streamingtv и streamingmovies) имеют  весьма высокую корреляцию между собой (\(0.960473\)) и с ежемесячными платежами, подчеркивая популярность и влияние этих услуг на общую стоимость услуг для клиентов.



In [ ]:
display(phik_matrix_stayed)

In [ ]:
# Визуализация матрицы корреляции phi_k для клиентов, которые остались
plt.figure(figsize=(12, 10))
sns.heatmap(phik_matrix_stayed, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title("Phik Correlation Matrix for Stayed Customers")
plt.show()

Из таблицы корреляции для группы клиентов, которые остались, можно сделать следующие ключевые выводы:

1. **Продолжительность контракта** (\(0.844546\)) продолжает демонстрировать высокую корреляцию с общими расходами (totalcharges), что подтверждает, что клиенты с более длительными контрактами, как правило, накапливают больше расходов за время использования услуг. Это указывает на долгосрочную вовлеченность и возможно большую лояльность среди оставшихся клиентов.

2. **Ежемесячные платежи** (\(0.743925\)) имеют высокую  корреляцию с общими расходами, что свидетельствует о том, что регулярные ежемесячные платежи значительно способствуют общему объему расходов клиентов. Это подчеркивает важность управления ежемесячными платежами для поддержания удовлетворенности клиентов.

3. **Интернет-услуги** (\(0.484988\)) и **многоканальная связь** (multiplelines) (\(0.478160\)) показывают умеренную корреляцию с общими расходами, что может отражать популярность и ценность этих услуг среди оставшихся клиентов.

4. **Онлайн-безопасность** (\(0.522232\)), **техподдержка** (\(0.534114\)), и **защита устройства** (\(0.568247\))  показывают заметную корреляцию с общими расходами, подчеркивая, что дополнительные услуги в области безопасности и поддержки важны для клиентов, что может способствовать их решению остаться.

5. **Партнер** (\(0.411492\)) показывает умеренную корреляцию с общими расходами, что может указывать на то, что наличие партнера влияет на решение остаться с компанией, возможно, из-за совместного использования услуг или лучших условий для семейных планов.

6. **Стриминговые услуги** (streamingtv и streamingmovies) демонстрируют весьма высокую корреляцию друг с другом и с ежемесячными платежами, что подтверждает их популярность и вклад в общую стоимость услуг среди оставшихся клиентов.


Сравнительный вывод корреляций между ушедшими и оставшимися клиентами:
    
Сравнивая корреляции между ушедшими и оставшимися клиентами, можно выявить несколько ключевых различий и сходств, которые помогают лучше понять факторы, влияющие на отток клиентов:

Сходства:
- **Продолжительность контракта** и **общие расходы** показывают высокую корреляцию в обеих группах, что подчеркивает их важность в контексте общей вовлеченности клиентов и накопления расходов во времени. Это указывает на то, что независимо от решения оставаться или уйти, эти два фактора имеют значительное влияние на финансовые взаимоотношения клиентов с компанией.

Различия:
- **Стриминговые услуги** (streamingtv и streamingmovies) и **дополнительные услуги** (онлайн-безопасность, техподдержка, защита устройства) имеют более высокие корреляции с общими расходами и ежемесячными платежами среди оставшихся клиентов по сравнению с ушедшими. Это может свидетельствовать о том, что удовлетворенность этими услугами и восприятие их ценности влияют на решение клиентов остаться.
- **Партнер** имеет более выраженную корреляцию с общими расходами среди оставшихся клиентов, что может указывать на то, что наличие партнера и совместное использование услуг способствует удержанию клиентов.
- **Интернет-услуги** и **многоканальная связь** (multiplelines) также показывают более высокую корреляцию с общими расходами и ежемесячными платежами среди оставшихся клиентов, подчеркивая их роль в создании добавленной стоимости для клиентов, решивших остаться.
 Заключение:
- **Долгосрочные отношения и дополнительные услуги** играют ключевую роль в удержании клиентов. Оставшиеся клиенты, как правило, более вовлечены и ценят широкий спектр услуг, что отражается в их более высоких общих и ежемесячных расходах.
- **Предпочтения в услугах** (например, стриминговые сервисы и интернет-услуги) и **способы оплаты** могут влиять на решение клиентов остаться, что подчеркивается различиями в корреляциях этих признаков между двумя группами.


# **Шаг 5 Подготовка данных**

In [ ]:
df.info()

In [ ]:
print(df['seniorcitizen'].value_counts())

In [ ]:
print(df['seniorcitizen'].dtype)

In [ ]:
df['seniorcitizen'] = df['seniorcitizen'].astype(str)

Преобразование seniorcitizen в строку: Это шаг необходим для унификации типов данных перед их обработкой. Поскольку seniorcitizen изначально является числовым признаком, представляющим бинарную категорию (0 или 1), его преобразование в строковый формат позволяет более эффективно применять к нему методы кодирования категориальных признаков, такие как One-Hot Encoding.

Установка customerid в качестве индекса

In [ ]:
df.set_index('customerid', inplace=True)

In [ ]:
# Assuming df_final DataFrame and 'churn' is the target variable
X = df.drop(['churn'], axis=1)  # Drop target variable
y = df['churn']

In [ ]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

# Identifying numeric and categorical features
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Creating transformers for the preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Scaling numeric features
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))  # One-hot encoding categorical features, dropping the first column
])

# Combining transformers into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Applying the transformations to the training and test sets
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Extracting new column names for the transformed features
new_numeric_features = numeric_features.tolist()
# new_categorical_features = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
new_categorical_features = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out()

# Combining all new feature names
all_new_features = new_numeric_features + new_categorical_features.tolist()

# Convert the transformed features back to DataFrame for readability (optional)
X_train_transformed_df = pd.DataFrame(X_train_transformed, columns=all_new_features, index=X_train.index)
X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=all_new_features, index=X_test.index)



In [ ]:
display(X_train_transformed_df.head())

In [ ]:
X_train_transformed_df.info()

Данные для обучения модели подготовлены. Данные были разделены на две выборки, проведено масштабирование числовых признаков и кодирование категориальных с учетом особенностей данных и моделей.

В процессе подготовки данных для моделирования были выполнены следующие ключевые шаги:

1. **Преобразование переменной seniorcitizen в строковый тип**: Это позволяет обрабатывать бинарный признак как категориальный, упрощая последующее кодирование и анализ.

2. **Разделение на обучающую и тестовую выборки**: Использование 75% данных для обучения и 25% для тестирования обеспечивает баланс между тренировкой модели и оценкой ее производительности на новых данных.

3. **Идентификация и обработка признаков**:
   - **Числовые признаки** масштабируются с помощью StandardScaler для улучшения производительности модели, нормализуя распределение данных.
   - **Категориальные признаки** преобразуются через OneHotEncoder, что позволяет модели корректно интерпретировать категории без предположений о их порядке или величине.

4. **Применение предобработки**: Обучение преобразований на обучающей выборке и их применение к тестовой выборке гарантирует, что данные обрабатываются правильно, что критически важно для точности модели.

Эти шаги необходимы для эффективной подготовки данных, что обеспечивает адекватность и точность последующего моделирования.

# **Шаг 6: Машинное обучение моделей**

Training model CatBoost

In [ ]:
param_dist = {
     'iterations': [100, 500, 1000],
    'learning_rate': uniform(0.01, 0.3),
    'depth': [4, 6, 8, 10]}


# Inisialization model CatBoostClassifier
catboost_model = CatBoostClassifier(verbose=False, eval_metric='AUC', loss_function='Logloss', random_seed=RANDOM_STATE)

#  RandomizedSearchCV
random_search = RandomizedSearchCV(catboost_model, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, random_state=RANDOM_STATE, verbose=2)

# fit model with RandomizedSearchCV
random_search.fit(X_train_transformed_df, y_train)

In [ ]:
print(f"best params: {random_search.best_params_}")
print(f"best ROC AUC: {random_search.best_score_}")


best_model = random_search.best_estimator_

best params: {'depth': 4, 'iterations': 1000, 'learning_rate': 0.11187815841373282}
best ROC AUC: 0.9056035506322976

In [ ]:
# Creating a logistic regression model
lr = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)

# Setting up the parameter grid for randomized search
param_dist = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Setting up the randomized search with cross-validation
random_search_lr = RandomizedSearchCV(lr, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, random_state=RANDOM_STATE)

# Fitting the model
random_search_lr.fit(X_train_transformed, y_train)




In [ ]:
# Best parameters and best ROC AUC score
print(f"best params_lr: {random_search_lr.best_params_}")
print(f"best ROC AUC_lr: {random_search_lr.best_score_}")


best_model_lr = random_search_lr.best_estimator_



best params_lr: {'solver': 'sag', 'C': 100}
best ROC AUC_lr: 0.7704748384435064



Были обучены модели CatBoost и логистическая регрессия. CatBoost показал лучшие результаты на обучающем наборе по метрике ROC_AUC -лучшие параметры:  {'depth': 4, 'iterations': 1000, 'learning_rate': 0.11187815841373282} лучший ROC AUC: 0.9056035506322976. Поэтому лучше использовать эти параметры для тестирования на тестовом образце.

# **Шаг 7. Выбор лучшей модели**

In [ ]:
#testing on test set
y_pred = best_model.predict(X_test_transformed_df)
y_pred_proba = best_model.predict_proba(X_test_transformed_df)[:, 1]

#  ROC AUC and Accuracy
roc_auc_test = roc_auc_score(y_test, y_pred_proba)
accuracy_test = accuracy_score(y_test, y_pred)

roc_auc_test, accuracy_test

roc_auc - 0.92936553813648, accuracy_test - 0.9346961953435547

В результате использования модели CatBoost для классификации и последующей оптимизации ее параметров с помощью рандомизированного поиска на тестовом наборе данных были достигнуты следующие результаты:

ROC AUC: 0.928. Accuracy: 0.935. Эти результаты указывают на высокую эффективность модели CatBoost в прогнозировании целевой переменной. Значение ROC AUC указывает на высокую способность модели различать классы. Уровень точности, близкий к 93.5%, также подчеркивает, что модель правильно классифицирует большинство примеров.

Эти показатели могут свидетельствовать о том, что модель CatBoost успешно справилась с задачей, обеспечив высокую точность и возможность обобщения на незнакомых данных. Это делает CatBoost отличным выбором для задач классификации в этом контексте.

In [ ]:
#визуализация ROC-кривой

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

Значения AUC около 0.93 указывает на способность модели превосходить случайное угадывание .
ROC-кривая модели находится выше диагональной линии, это означает, что модель имеет лучшую производительность, чем случайное угадывание.


In [ ]:
#матрица ошибок

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
cm_df = pd.DataFrame(cm, index=["Actual Negative", "Actual Positive"],
                     columns=["Predicted Negative", "Predicted Positive"])
print("\nМатрица ошибок в формате DataFrame:")
print(cm_df)

Из матрицы ошибок можно сделать следующие выводы относительно производительности модели классификации:

Истинно отрицательные (TN): Модель правильно предсказала 1476 случаев, когда клиенты не ушли (Actual Negative), что указывает на хорошую способность модели идентифицировать клиентов, склонных оставаться.  
Ложно положительные (FP): В 16 случаях модель неверно предсказала, что клиенты уйдут (Predicted Positive), когда на самом деле они остались (Actual Negative). Это может указывать на излишнюю чувствительность модели к некоторым признакам, ведущую к ошибочному определению оттока.  
Ложно отрицательные (FN): Модель пропустила 98 случаев, когда клиенты ушли (Actual Positive), предсказав, что они останутся (Predicted Negative). Это один из наиболее критичных типов ошибок, так как пропущенные случаи оттока могут привести к потере дохода и клиентской базы.  
Истинно положительные (TP): Модель корректно идентифицировала 171 случай, когда клиенты ушли (Actual Positive). Это говорит о том, что модель эффективно выявляет клиентов, склонных к оттоку.  
<b>Общий анализ:</b>  
Модель демонстрирует высокую способность правильно классифицировать отрицательные случаи (клиенты остаются), что подтверждается большим количеством истинно отрицательных результатов по сравнению с ложно положительными.
Количество ложно отрицательных результатов является зоной для улучшения. Снижение этого числа позволит модели более эффективно выявлять реальные случаи оттока.  

In [ ]:
#Анализ важности признаков

feature_importances = best_model.get_feature_importance()
plt.figure(figsize=(10, 8))
sns.barplot(y=X_train_transformed_df.columns, x=feature_importances)
plt.title('Feature Importances')
plt.show()

Наиболее важными признаками являются - длительность контракта , следом идут общие и помесячные траты. Эти Признаки являются ключевыми для принятия решений моделью.

##Сохранение модели

In [ ]:
#saving model
# naming model file
filename = 'finalized_model.sav'

with open(filename, 'wb') as file:
    # saving model to file
    pickle.dump(best_model, file)

# **Шаг 8. Заключение и рекомендации для заказчика**

Основной целью этого проекта была разработка прогнозной модели для «ТелеДом», телекоммуникационного оператора, стремящегося сократить отток клиентов, предлагая акции и специальные условия тем, кто планирует прекратить свои услуги. Цель и задачи проекта выполнены.

Проект начался с загрузки предоставленных наборов данных и проведения первоначальной проверки для понимания структуры, качества и типов доступной информации. Для каждого кадра данных был проведен углубленный исследовательский анализ с последующей необходимой предварительной обработкой данных. Этот этап включал выявление и обработку пропущенных значений, выбросов (менее 3 %) и обеспечение того, чтобы общее качество данных было подходящим для моделирования. Анализ позволил получить представление о доступных функциях и их потенциальной полезности для прогнозирования оттока клиентов.

Выбранные функции были объединены в единый фрейм данных с использованием ключа для унификации данных, что позволило более упростить процесс анализа и моделирования.

Был выполнен дальнейший исследовательский анализ объединенного кадра данных, визуализация распределения признаков и выполнение корреляционного анализа для понимания взаимосвязей между переменными. Все корреляции с оттоком не были сильными. Наиболее значимыми результатами являются продолжительность контракта (0,375). Такая положительная корреляция предполагает, что более длительный срок контракта связан с более высоким уровнем оттока клиентов. Это может указывать на проблемы с обязательствами клиентов или неудовлетворенность ими с течением времени.

Общие расходы (0,303): демонстрирует умеренную положительную корреляцию с оттоком. Более высокие общие расходы могут привести к более высокому оттоку, возможно, из-за воспринимаемой ценности или финансовых ограничений с точки зрения клиента. Партнер (0,227) и «Ежемесячные сборы» (0,226): оба демонстрируют одинаковый уровень положительной корреляции с оттоком. Наличие партнера и более высокие ежемесячные платежи умеренно способствуют оттоку клиентов. Это может отразиться на финансовом бремени и динамике принятия решений в домохозяйствах.

Данные были подготовлены для обучения модели, которое включало разделение на обучающий и тестовый наборы, масштабирование числовых характеристик и кодирование категориальных переменных. Эта подготовка была адаптирована с учетом требований прогнозных моделей и конкретных характеристик данных.

Было обучено несколько моделей (CatBoost и логистическа регрессия, причем обе модели подверглись настройке гиперпараметров для оптимизации их производительности. Этот процесс включал оценку различных алгоритмов и их конфигураций, чтобы найти наиболее эффективное решение для прогнозирования оттока клиентов.

После обучения была выбрана наиболее эффективная модель на основе ее показателей производительности, включая  (ROC AUC). Затем выбранная модель была протестирована на отдельном наборе тестовых данных, чтобы подтвердить ее прогностическую способность и обобщаемость.

В рамках проекта была успешно разработана прогнозирующая модель, способная с высокой точностью идентифицировать клиентов, которые могут уйти. Самая эффективная модель CatBoost достигла значения ROC AUC 0,928  на тестовом наборе, что указывает на ее хорошую способность точно классифицировать клиентов.

**Рекомендации Заказчику**

По результатам проекта «ТелеДом» рекомендуется внедрить прогнозную модель для превентивного выявления клиентов из группы риска. Поступая таким образом, компания может ориентировать этих людей на специальные предложения, рекламные акции или персонализированные коммуникационные стратегии, направленные на сохранение их бизнеса. Учитывая результаты корреляции, вот краткие рекомендации для «ТелеДома» по снижению оттока клиентов:

- Приоритизируйте продолжительность контракта. Более длительные контракты с меньшей вероятностью будут отброшены. Предложение стимулов для более длительных обязательств может снизить уровень оттока сотрудников.

- Пересмотрите цены и сборы. Высокие общие и ежемесячные расходы связаны с более высоким оттоком клиентов. Рассмотрите возможность пересмотра стратегий ценообразования, особенно для дорогостоящих планов, чтобы убедиться, что они предлагают оптимальное соотношение цены и качества.

- Сосредоточив внимание на этих направлениях, «ТелеДом» может реализовать целевые стратегии, позволяющие эффективно снизить отток клиентов и повысить общую удовлетворенность и лояльность клиентов.

# Загрузка модели и предсказание для случайного пользователя

##Подготовка данных для предсказания

В этом разделе будут произведены следующие шаги
*   Создание исходных таблиц и заполнение случайными данными одного абонента
*   Предобработка для всех датасетов, такие же как на исходных данных
*   Соединение датафреймов в общий датафрейм
*   Выполнения предобработки и трансформации точно также как и на исходном общем датафрейме
*   Загрузка модели
*   Предсказание для этих данных








In [ ]:
#creating new datasets from the data
contract_test = pd.read_csv('/content/drive/MyDrive/datasets/contract_new.csv')
internet_test = pd.read_csv('/content/drive/MyDrive/datasets/internet_new.csv')
phone_test=pd.read_csv('/content/drive/MyDrive/datasets/phone_new.csv')
personal_test=pd.read_csv('/content/drive/MyDrive/datasets/personal_new.csv')

In [ ]:
# # Seed for reproducibility
# np.random.seed(42)  # Setting the random seed
rng = np.random.default_rng(RANDOM_STATE)

In [ ]:
#Finding the intersection of unique customerIDs across all dataframes to ensure the selected customerID is present in each dataframe
common_ids = set(contract_test['customerID'].unique()) & set(internet_test['customerID'].unique()) & \
             set(phone_test['customerID'].unique()) & set(personal_test['customerID'].unique())

In [ ]:
# Selecting a random customerID from the intersection
# random_id = np.random.choice(list(common_ids))
random_id = rng.choice(list(common_ids))

In [ ]:
# Filtering each dataframe for the selected customerID
contract_test = contract_test[contract_test['customerID'] == random_id]
internet_test = internet_test[internet_test['customerID'] == random_id]
phone_test = phone_test[phone_test['customerID'] == random_id]
personal_test = personal_test[personal_test['customerID'] == random_id]

In [ ]:
contract_test.head()

In [ ]:
internet_test.head()

In [ ]:
phone_test.head()

In [ ]:
personal_test.head()

In [ ]:
#function for preprosessing contract datatset

def preprocess_data(df):
    """
    Preprocess the data with the following steps:
    - Convert column names to lowercase
    - Remove duplicates
    - Convert 'begindate' and 'enddate' to datetime
    - Convert 'totalcharges' to numeric, replace non-numeric with NaN, then missing values with 0
    - Create a 'churn' column based on 'enddate'
    - Replace missing 'enddate' with a specific date

    Args:
    df (pd.DataFrame): Input dataframe with contract information

    Returns:
    pd.DataFrame: Preprocessed dataframe
    """
    # Convert column names to lowercase
    df.columns = df.columns.str.lower()

    # Remove duplicates
    df.drop_duplicates(inplace=True)

    # Convert 'begindate' and 'enddate' to datetime
    df['begindate'] = pd.to_datetime(df['begindate'])
    df['enddate'] = pd.to_datetime(df['enddate'], errors='coerce')

    # Convert 'totalcharges' to numeric, replace non-numeric with NaN, then fill NaN with 0
    df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce').fillna(0)

    # Create a 'churn' column based on 'enddate'
    df['churn'] = df['enddate'].notna().astype(int)

    # Replace missing 'enddate' for non-churned customers with a specific date
    replacement_date = pd.Timestamp('2020-02-01')
    df['enddate'] = df['enddate'].fillna(replacement_date)

    return df


In [ ]:
#preprosessing contract datatset
contract_test = preprocess_data(contract_test)
contract_test.head()

In [ ]:
def preprocess_others(df):
    """
    Preprocess the data by converting column names to lowercase.

    Args:
    df (pd.DataFrame): Input dataframe to preprocess.

    Returns:
    pd.DataFrame: Preprocessed dataframe with lowercase column names.
    """
    df.columns = df.columns.str.lower()
    return df

In [ ]:
#preprosessing other datasets
internet_test = preprocess_others(internet_test)
personal_test = preprocess_others(personal_test)
phone_test = preprocess_others(phone_test)

In [ ]:
internet_test.head()

In [ ]:
personal_test.head()

In [ ]:
phone_test.head()

In [ ]:
def merge_dataframes(contract, personal, internet, phone):
    """
    Merge dataframes on 'customerid' with an outer join to include all records.

    Args:
    contract (pd.DataFrame): Contract information dataframe.
    personal (pd.DataFrame): Personal information dataframe.
    internet (pd.DataFrame): Internet service information dataframe.
    phone (pd.DataFrame): Phone service information dataframe.

    Returns:
    pd.DataFrame: Merged dataframe containing all information.
    """
    # Merge contract with personal
    merged_df = contract.merge(personal, on='customerid', how='outer')

    # Add data from internet
    merged_df = merged_df.merge(internet, on='customerid', how='outer')

    # Add data from phone
    final_df = merged_df.merge(phone, on='customerid', how='outer')

    return final_df


In [ ]:
def preprocess_merged_data(df):
    """
    Preprocess the merged dataframe by filling missing values, converting data types,
    removing duplicates, and calculating new columns.

    Args:
    df (pd.DataFrame): Merged dataframe to preprocess.

    Returns:
    pd.DataFrame: Preprocessed dataframe ready for analysis or modeling.
    """
    # Calculate the duration of the contract and add a new column
    df['contract_duration'] = (df['enddate'] - df['begindate']).dt.days

    # Select columns of type object or float for further processing
    cols = df.select_dtypes(include=['object', 'float']).columns.tolist()

    # Manually identified columns to include
    additional_cols = ['churn', 'contract_duration', 'seniorcitizen']

    # Combine lists while ensuring uniqueness
    cols = list(set(cols + additional_cols))

    # Create a copy of the dataframe with selected columns
    df = df[cols].copy()

    # Remove duplicates
    df.drop_duplicates(keep=False, inplace=True)

    # Filling missing values for 'multiplelines'
    df['multiplelines'].fillna('Not Applicable', inplace=True)

    # Assume missing values indicate no internet service for specific columns
    internet_service_columns = [
        'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection',
        'techsupport', 'streamingtv', 'streamingmovies'
    ]

    # Fill missing values with 'Not Applicable' for internet service columns
    df[internet_service_columns] = df[internet_service_columns].fillna('Not Applicable')

    # Convert 'seniorcitizen' to string for consistency
    df['seniorcitizen'] = df['seniorcitizen'].astype(str)

    # Set 'customerid' as the index of the dataframe
    df.set_index('customerid', inplace=True)

    return df


In [ ]:
merged_df = merge_dataframes(contract_test, personal_test, internet_test, phone_test)
preprocessed_df = preprocess_merged_data(merged_df)

In [ ]:
merged_df.info()

In [ ]:
preprocessed_df.head()

In [ ]:
print(preprocessed_df.info())


In [ ]:
preprocessed_df.info()

In [ ]:
# Assuming df_final DataFrame and 'churn' is the target variable
X = preprocessed_df.drop(['churn'], axis=1)  # Drop target variable
y = preprocessed_df['churn']

In [ ]:
# preprosessing
X_new_transformed = preprocessor.transform(X)


new_features = new_numeric_features + new_categorical_features.tolist()


X_new_transformed_df = pd.DataFrame(X_new_transformed, columns=new_features)

In [ ]:
X_new_transformed_df.index = preprocessed_df.index

In [ ]:
X_new_transformed_df.head()

In [ ]:
X_new_transformed_df.info()

In [ ]:
# loading model from file
with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)

In [ ]:
y_pred = loaded_model.predict(X_new_transformed)


In [ ]:

# Printing model predictions for the new data
print("Model predictions for new data:")
print(y_pred)

In [ ]:
# finging real value for this user
real_value = df.loc[random_id, 'churn']

# print real value
print(f"Real value in original df {random_id}: {real_value}")

Загрузка модели и предсказание для случайного абонента успешно выполнены.